<a href="https://colab.research.google.com/github/Edna-soso/DataAnalysis_Project/blob/main/Data_Prepro/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
from google.colab import files

In [66]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/python_OP/OnlineRetail.csv', encoding = 'unicode_escape')

print(data.shape)

(541909, 8)


In [68]:
class data_preprocessing:
  def __init__(self, data):
    self.data = data
  def True_Data(self):
    data.isna().sum()
    self.data = data[pd.notnull(data['CustomerID'])]
    self.data = data.query("Quantity > 0")
  def Add_Colunm(self):
    data['Date']=[item[0] for item in data['InvoiceDate'].str.split()]
    data['Time']=[item[1] for item in data['InvoiceDate'].str.split()]
    data['Hour']=[item[0] for item in data['Time'].str.split(':')]
    data['Month']=[item[0] for item in data['Date'].str.split('/')]
    data['TotalCost']=data['Quantity']*data['UnitPrice']
  def Data(self):
    return data



In [69]:
df = data_preprocessing(data)
df.True_Data()
df.Add_Colunm()
df.Data()



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,Time,Hour,Month,TotalCost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,12/1/2010,8:26,8,12,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,12/1/2010,8:26,8,12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,12/1/2010,8:26,8,12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,12/1/2010,8:26,8,12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,12/1/2010,8:26,8,12,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,12/9/2011,12:50,12,12,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12/9/2011,12:50,12,12,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,12/9/2011,12:50,12,12,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,12/9/2011,12:50,12,12,16.60


In [65]:
df.data_df().to_csv('Clean_Dataset.csv') 
files.download('Clean_Dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>